In [21]:
import pandas as pd
from geopy.distance import distance
from tqdm.auto import tqdm

In [22]:
FRONT_YONGE_FLOW_COLLECTION_POINT = (43.646882, -79.376952)
front_count_df = pd.read_csv("../../raw_data/traffic_volume/traffic_volume_raw-data-2020-2029.zip")

In [23]:
front_count_df["time_start"] = pd.to_datetime(front_count_df["time_start"], utc=True)
front_count_df["time_start"] = front_count_df["time_start"].apply(lambda x: x.to_datetime64())
front_count_df["time_end"] = pd.to_datetime(front_count_df["time_end"], utc=True)
front_count_df["time_end"] = front_count_df["time_end"].apply(lambda x: x.to_datetime64())

In [24]:
front_count_df = front_count_df.drop(
    columns=["count_id", "count_date", "location_id", "centreline_type", "centreline_id", "px"])
front_count_df = front_count_df.drop(columns=list(
    filter(lambda x: "peds" in x, front_count_df.columns)))
front_count_df = front_count_df.drop(columns=list(
    filter(lambda x: "bike" in x, front_count_df.columns)))
front_count_df = front_count_df.drop(columns=list(
    filter(lambda x: "other" in x, front_count_df.columns)))

In [25]:
front_count_car_columns = list(filter(lambda x: "car" in x, front_count_df.columns))
front_count_truck_columns = list(filter(lambda x: "truck" in x, front_count_df.columns))
front_count_bus_columns = list(filter(lambda x: "bus" in x, front_count_df.columns))

In [26]:
front_count_df["car_count"] = front_count_df[front_count_car_columns].sum(axis=1)
front_count_df["truck_count"] = front_count_df[front_count_truck_columns].sum(axis=1)
front_count_df["bus_count"] = front_count_df[front_count_bus_columns].sum(axis=1)

In [27]:
front_count_df = front_count_df.drop(columns=front_count_car_columns)
front_count_df = front_count_df.drop(columns=front_count_truck_columns)
front_count_df = front_count_df.drop(columns=front_count_bus_columns)

In [28]:
count_cols = list(filter(lambda x: "count" in x, front_count_df.columns))
front_count_df["total_count"] = front_count_df[count_cols].sum(axis=1)
front_count_df = front_count_df.drop(columns=["car_count", "truck_count", "bus_count"])

In [29]:
tqdm.pandas()
front_count_df["distance_from_point"] = front_count_df.progress_apply(
    lambda x: distance((x["lat"], x["lng"]), FRONT_YONGE_FLOW_COLLECTION_POINT).km, axis=1)

  0%|          | 0/123075 [00:00<?, ?it/s]

In [30]:
front_count_df = front_count_df.sort_values(by="distance_from_point")

In [31]:
front_count_df

,_id,location,lng,lat,time_start,time_end,total_count,distance_from_point
26376,26377,FRONT ST AT YONGE ST (PX 29),-79.376974,43.646884,2020-01-25 08:45:00,2020-01-25 09:00:00,198,0.001789
79746,79747,FRONT ST AT YONGE ST (PX 29),-79.376974,43.646884,2022-02-26 17:45:00,2022-02-26 18:00:00,301,0.001789
79747,79748,FRONT ST AT YONGE ST (PX 29),-79.376974,43.646884,2022-02-23 07:30:00,2022-02-23 07:45:00,294,0.001789
79748,79749,FRONT ST AT YONGE ST (PX 29),-79.376974,43.646884,2022-02-23 07:45:00,2022-02-23 08:00:00,328,0.001789
79749,79750,FRONT ST AT YONGE ST (PX 29),-79.376974,43.646884,2022-02-23 08:00:00,2022-02-23 08:15:00,322,0.001789
...,...,...,...,...,...,...,...,...
70017,70018,REESOR RD AT STEELES AVE E (EAST INTERSECTION),-79.199333,43.848510,2022-01-27 17:30:00,2022-01-27 17:45:00,342,26.580770
70018,70019,REESOR RD AT STEELES AVE E (EAST INTERSECTION),-79.199333,43.848510,2022-01-27 17:45:00,2022-01-27 18:00:00,330,26.580770
69987,69988,REESOR RD AT STEELES AVE E (EAST INTERSECTION),-79.199333,43.848510,2022-01-27 07:30:00,2022-01-27 07:45:00,301,26.580770
70000,70001,REESOR RD AT STEELES AVE E (EAST INTERSECTION),-79.199333,43.848510,2022-01-27 11:15:00,2022-01-27 11:30:00,185,26.580770


In [32]:
front_count_df_closest = front_count_df[front_count_df["distance_from_point"] <= 2]

In [33]:
front_count_df_closest_front = front_count_df_closest[front_count_df_closest["location"].str.contains("FRONT")]

In [34]:
front_count_df_closest_front = front_count_df_closest_front.drop(columns=["_id", "time_start"])

In [35]:
front_count_df_closest_front_grouped = front_count_df_closest_front.groupby("time_end").mean().reset_index()

In [36]:
front_count_df_closest_front_grouped["hour"] = front_count_df_closest_front_grouped["time_end"].dt.hour
front_count_df_closest_front_grouped["minute"] = front_count_df_closest_front_grouped["time_end"].dt.minute
front_count_df_closest_front_grouped["day_of_week"] = front_count_df_closest_front_grouped["time_end"].dt.dayofweek

In [37]:
front_count_df_closest_front_grouped = front_count_df_closest_front_grouped.drop("time_end", axis=1)

In [38]:
front_count_df_closest_front_grouped

,lng,lat,total_count,distance_from_point,hour,minute,day_of_week
0,-79.368404,43.649997,284.076923,0.773430,7,45,3
1,-79.368404,43.649997,340.769231,0.773430,8,0,3
2,-79.368404,43.649997,358.307692,0.773430,8,15,3
3,-79.368404,43.649997,316.384615,0.773430,8,30,3
4,-79.368404,43.649997,327.769231,0.773430,8,45,3
...,...,...,...,...,...,...,...
411,-79.371465,43.649450,559.000000,0.526677,17,0,5
412,-79.371465,43.649450,594.000000,0.526677,17,15,5
413,-79.371465,43.649450,549.000000,0.526677,17,30,5
414,-79.371465,43.649450,552.000000,0.526677,17,45,5


In [39]:
front_count_df_closest_front_grouped.to_csv("../../csv/front_count_grouped_dow.csv.zip", index=False, compression="zip")

In [40]:
front_count_df_closest_front.to_csv("../../csv/front_count_closest.csv.zip", index=False, compression="zip")